In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib
from scipy.stats import gamma
import numpy as np
import c3aidatalake

# Consolidating all Factors into Single CSV File

* Proportion of all High-Risk Diseases
* Proportion of Unemployment
* Medical Preparedness
* Proportion of Race
* Social Distance Grade Score by Month
    * Will need to offset for incubation period (let's try without it first)
* Cases by Rural

## List of Official CFPB Listed Rural Counties

In [241]:
rural = pd.read_csv('final_rural_counties.csv').iloc[:,1:]
rural.head()

,fips_code,county,state,c3_data_id
0,51001.0,Accomack,Virginia,Accomack_Virginia_UnitedStates
1,19001.0,Adair,Iowa,Adair_Iowa_UnitedStates
2,21001.0,Adair,Kentucky,Adair_Kentucky_UnitedStates
3,29001.0,Adair,Missouri,Adair_Missouri_UnitedStates
4,40001.0,Adair,Oklahoma,Adair_Oklahoma_UnitedStates


----

# High Risk Disease

In [8]:
hrd = pd.read_csv('all_county_disease.csv')
hrd.head(), hrd.shape

(  COUNTY_NAME STATE_NAME  FIPS  county_pop2018_18 and older  \
 0     Autauga    Alabama  1001                        42438   
 1     Baldwin    Alabama  1003                       170912   
 2     Barbour    Alabama  1005                        19689   
 3        Bibb    Alabama  1007                        17813   
 4      Blount    Alabama  1009                        44448   
 
    anycondition_number  Obesity_number  Heart disease_number  COPD_number  \
 0                20181           15193                  3345         3644   
 1                68790           50761                 13414        14692   
 2                11325            8013                  2159         2373   
 3                 9190            6894                  1533         1789   
 4                20584           15112                  4101         4661   
 
    diabetes_number  CKD_number  proportionAnyCondition  proportionObese  \
 0             5462        1326                0.475541         0.35

In [15]:
## We will be using any condition because the conditions 
## are not mutually exclusive (ex. can be obeses and also have heart disease, so we want to avoid excess counts)

## Isolating only rural counties
hrd_rural = hrd.loc[hrd['FIPS'].isin(list(rural['fips_code']))]
rural_conditions_proportion = hrd_rural.loc[:, ['COUNTY_NAME','STATE_NAME','FIPS', 'proportionAnyCondition']]
rural_conditions_proportion

,COUNTY_NAME,STATE_NAME,FIPS,proportionAnyCondition
2,Barbour,Alabama,1005,0.575194
5,Bullock,Alabama,1011,0.606549
6,Butler,Alabama,1013,0.581035
9,Cherokee,Alabama,1019,0.559310
11,Choctaw,Alabama,1023,0.571929
...,...,...,...,...
3137,Sweetwater,Wyoming,56037,0.402884
3138,Teton,Wyoming,56039,0.242862
3139,Uinta,Wyoming,56041,0.422068
3140,Washakie,Wyoming,56043,0.375082


----

# Unemployment Rate

In [447]:
raw_c3_data = pd.read_csv('raw_counties_info.csv')
raw_c3_data

,hospitalIcuBeds,hospitalStaffedBeds,hospitalLicensedBeds,latestTotalPopulation,populationOfAllChildren,latestLaborForce,latestEmployedPopulation,latestUnemployedPopulation,latestUnemploymentRate,laborForceOfAllChildren,locationType,populationCDS,id,name,typeIdent,location.value.id,location.timestamp,fips.id
0,6.0,25.0,25.0,24527.0,24527.0,10090.0,9633.0,457.0,4.529237,10090.0,county,24527.0,Abbeville_SouthCarolina_UnitedStates,Abbeville,EP_LOC,Abbeville_SouthCarolina_UnitedStates,2020-11-11T00:00:00Z,45001.0
1,7.0,171.0,186.0,62045.0,62045.0,24074.0,22369.0,1705.0,7.082329,24074.0,county,62045.0,Acadia_Louisiana_UnitedStates,Acadia,EP_LOC,Acadia_Louisiana_UnitedStates,2020-11-11T00:00:00Z,22001.0
2,10.0,46.0,52.0,32316.0,32316.0,16001.0,15096.0,905.0,5.655897,16001.0,county,32316.0,Accomack_Virginia_UnitedStates,Accomack,EP_LOC,Accomack_Virginia_UnitedStates,2020-11-11T00:00:00Z,51001.0
3,97.0,1195.0,1305.0,481587.0,481587.0,263531.0,248015.0,15516.0,5.887732,263531.0,county,481587.0,Ada_Idaho_UnitedStates,Ada,EP_LOC,Ada_Idaho_UnitedStates,2020-11-11T00:00:00Z,16001.0
4,4.0,25.0,25.0,7152.0,7152.0,3862.0,3747.0,115.0,2.977732,3862.0,county,7152.0,Adair_Iowa_UnitedStates,Adair,EP_LOC,Adair_Iowa_UnitedStates,2020-11-11T00:00:00Z,19001.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3147,42.0,447.0,456.0,213787.0,213787.0,97557.0,80938.0,16619.0,17.035169,97557.0,county,213787.0,Yuma_Arizona_UnitedStates,Yuma,EP_LOC,Yuma_Arizona_UnitedStates,2020-11-11T00:00:00Z,4027.0
3148,6.0,30.0,30.0,10019.0,10019.0,5010.0,4868.0,142.0,2.834331,5010.0,county,10019.0,Yuma_Colorado_UnitedStates,Yuma,EP_LOC,Yuma_Colorado_UnitedStates,2020-11-11T00:00:00Z,8125.0
3149,NaN,NaN,NaN,14179.0,14179.0,4947.0,4268.0,679.0,13.725490,4947.0,county,14179.0,Zapata_Texas_UnitedStates,Zapata,EP_LOC,Zapata_Texas_UnitedStates,2020-11-11T00:00:00Z,48505.0
3150,NaN,NaN,NaN,11840.0,11840.0,3309.0,2748.0,561.0,16.953762,3309.0,county,11840.0,Zavala_Texas_UnitedStates,Zavala,EP_LOC,Zavala_Texas_UnitedStates,2020-11-11T00:00:00Z,48507.0


In [448]:
rural_c3 = rural.merge(raw_c3_data, left_on = 'fips_code', right_on = 'fips.id')
rural_unemp_df = rural_c3[['fips_code','latestUnemploymentRate']]
rural_unemp_df

,fips_code,latestUnemploymentRate
0,51001.0,5.655897
1,19001.0,2.977732
2,21001.0,5.503603
3,29001.0,2.987000
4,40001.0,4.933655
...,...,...
1614,48503.0,5.745519
1615,2290.0,8.801415
1616,8125.0,2.834331
1617,48507.0,16.953762


----

# Medical Preparedness

To measure medical preparedness,  We will take a look at ICU Beds, not licensed beds. Licensed beds are all the of the beds that a hospital is licensed to have. This number will include beds reserved for non-covid patients. But because we want to know how prepared a county is for COVID medical preparedness we need to know how many ICU beds they have in total. Not every county in the US has a hospital, checked and referenced a 2020 [Kaiser Health Network Report](https://khn.org/news/as-coronavirus-spreads-widely-millions-of-older-americans-live-in-counties-with-no-icu-beds/) which discussed a lack of ICU Beds.

In [451]:
icu_zeros = rural_c3.fillna(value = 0)
icu_zeros

,fips_code,county,state,c3_data_id,hospitalIcuBeds,hospitalStaffedBeds,hospitalLicensedBeds,latestTotalPopulation,populationOfAllChildren,latestLaborForce,...,latestUnemploymentRate,laborForceOfAllChildren,locationType,populationCDS,id,name,typeIdent,location.value.id,location.timestamp,fips.id
0,51001.0,Accomack,Virginia,Accomack_Virginia_UnitedStates,10.0,46.0,52.0,32316.0,32316.0,16001.0,...,5.655897,16001.0,county,32316.0,Accomack_Virginia_UnitedStates,Accomack,EP_LOC,Accomack_Virginia_UnitedStates,2020-11-11T00:00:00Z,51001.0
1,19001.0,Adair,Iowa,Adair_Iowa_UnitedStates,4.0,25.0,25.0,7152.0,7152.0,3862.0,...,2.977732,3862.0,county,7152.0,Adair_Iowa_UnitedStates,Adair,EP_LOC,Adair_Iowa_UnitedStates,2020-11-11T00:00:00Z,19001.0
2,21001.0,Adair,Kentucky,Adair_Kentucky_UnitedStates,0.0,0.0,0.0,19202.0,19202.0,6523.0,...,5.503603,6523.0,county,19202.0,Adair_Kentucky_UnitedStates,Adair,EP_LOC,Adair_Kentucky_UnitedStates,2020-11-11T00:00:00Z,21001.0
3,29001.0,Adair,Missouri,Adair_Missouri_UnitedStates,10.0,51.0,93.0,25343.0,25343.0,10077.0,...,2.987000,10077.0,county,25343.0,Adair_Missouri_UnitedStates,Adair,EP_LOC,Adair_Missouri_UnitedStates,2020-11-11T00:00:00Z,29001.0
4,40001.0,Adair,Oklahoma,Adair_Oklahoma_UnitedStates,5.0,30.0,67.0,22194.0,22194.0,8290.0,...,4.933655,8290.0,county,22194.0,Adair_Oklahoma_UnitedStates,Adair,EP_LOC,Adair_Oklahoma_UnitedStates,2020-11-11T00:00:00Z,40001.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1614,48503.0,Young,Texas,Young_Texas_UnitedStates,8.0,50.0,50.0,18010.0,18010.0,7867.0,...,5.745519,7867.0,county,18010.0,Young_Texas_UnitedStates,Young,EP_LOC,Young_Texas_UnitedStates,2020-11-11T00:00:00Z,48503.0
1615,2290.0,Yukon-Koyukuk,Alaska,Yukon-Koyukuk_Alaska_UnitedStates,0.0,0.0,0.0,5230.0,5230.0,2261.0,...,8.801415,2261.0,county,5230.0,Yukon-Koyukuk_Alaska_UnitedStates,Yukon-Koyukuk Census Area,EP_LOC,Yukon-Koyukuk_Alaska_UnitedStates,2020-11-11T00:00:00Z,2290.0
1616,8125.0,Yuma,Colorado,Yuma_Colorado_UnitedStates,6.0,30.0,30.0,10019.0,10019.0,5010.0,...,2.834331,5010.0,county,10019.0,Yuma_Colorado_UnitedStates,Yuma,EP_LOC,Yuma_Colorado_UnitedStates,2020-11-11T00:00:00Z,8125.0
1617,48507.0,Zavala,Texas,Zavala_Texas_UnitedStates,0.0,0.0,0.0,11840.0,11840.0,3309.0,...,16.953762,3309.0,county,11840.0,Zavala_Texas_UnitedStates,Zavala,EP_LOC,Zavala_Texas_UnitedStates,2020-11-11T00:00:00Z,48507.0


In [452]:
icu_zeros['icu_beds_per_capita'] = icu_zeros['hospitalIcuBeds']/icu_zeros['populationCDS']
icu_zeros

,fips_code,county,state,c3_data_id,hospitalIcuBeds,hospitalStaffedBeds,hospitalLicensedBeds,latestTotalPopulation,populationOfAllChildren,latestLaborForce,...,laborForceOfAllChildren,locationType,populationCDS,id,name,typeIdent,location.value.id,location.timestamp,fips.id,icu_beds_per_capita
0,51001.0,Accomack,Virginia,Accomack_Virginia_UnitedStates,10.0,46.0,52.0,32316.0,32316.0,16001.0,...,16001.0,county,32316.0,Accomack_Virginia_UnitedStates,Accomack,EP_LOC,Accomack_Virginia_UnitedStates,2020-11-11T00:00:00Z,51001.0,0.000309
1,19001.0,Adair,Iowa,Adair_Iowa_UnitedStates,4.0,25.0,25.0,7152.0,7152.0,3862.0,...,3862.0,county,7152.0,Adair_Iowa_UnitedStates,Adair,EP_LOC,Adair_Iowa_UnitedStates,2020-11-11T00:00:00Z,19001.0,0.000559
2,21001.0,Adair,Kentucky,Adair_Kentucky_UnitedStates,0.0,0.0,0.0,19202.0,19202.0,6523.0,...,6523.0,county,19202.0,Adair_Kentucky_UnitedStates,Adair,EP_LOC,Adair_Kentucky_UnitedStates,2020-11-11T00:00:00Z,21001.0,0.000000
3,29001.0,Adair,Missouri,Adair_Missouri_UnitedStates,10.0,51.0,93.0,25343.0,25343.0,10077.0,...,10077.0,county,25343.0,Adair_Missouri_UnitedStates,Adair,EP_LOC,Adair_Missouri_UnitedStates,2020-11-11T00:00:00Z,29001.0,0.000395
4,40001.0,Adair,Oklahoma,Adair_Oklahoma_UnitedStates,5.0,30.0,67.0,22194.0,22194.0,8290.0,...,8290.0,county,22194.0,Adair_Oklahoma_UnitedStates,Adair,EP_LOC,Adair_Oklahoma_UnitedStates,2020-11-11T00:00:00Z,40001.0,0.000225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1614,48503.0,Young,Texas,Young_Texas_UnitedStates,8.0,50.0,50.0,18010.0,18010.0,7867.0,...,7867.0,county,18010.0,Young_Texas_UnitedStates,Young,EP_LOC,Young_Texas_UnitedStates,2020-11-11T00:00:00Z,48503.0,0.000444
1615,2290.0,Yukon-Koyukuk,Alaska,Yukon-Koyukuk_Alaska_UnitedStates,0.0,0.0,0.0,5230.0,5230.0,2261.0,...,2261.0,county,5230.0,Yukon-Koyukuk_Alaska_UnitedStates,Yukon-Koyukuk Census Area,EP_LOC,Yukon-Koyukuk_Alaska_UnitedStates,2020-11-11T00:00:00Z,2290.0,0.000000
1616,8125.0,Yuma,Colorado,Yuma_Colorado_UnitedStates,6.0,30.0,30.0,10019.0,10019.0,5010.0,...,5010.0,county,10019.0,Yuma_Colorado_UnitedStates,Yuma,EP_LOC,Yuma_Colorado_UnitedStates,2020-11-11T00:00:00Z,8125.0,0.000599
1617,48507.0,Zavala,Texas,Zavala_Texas_UnitedStates,0.0,0.0,0.0,11840.0,11840.0,3309.0,...,3309.0,county,11840.0,Zavala_Texas_UnitedStates,Zavala,EP_LOC,Zavala_Texas_UnitedStates,2020-11-11T00:00:00Z,48507.0,0.000000


In [453]:
min(icu_zeros['icu_beds_per_capita']), max(icu_zeros['icu_beds_per_capita'])

(0.0, 0.003714020427112349)

In [456]:
min(icu_zeros['hospitalIcuBeds']), max(icu_zeros['hospitalIcuBeds'])

(0.0, 85.0)

There is very clearly a shortage of beds available per person in the US. And Not very many ICU beds prepared in general.

In [469]:
rural_icus = icu_zeros[['fips_code', 'hospitalIcuBeds', 'icu_beds_per_capita', 'populationCDS']]
rural_icus

,fips_code,hospitalIcuBeds,icu_beds_per_capita,populationCDS
0,51001.0,10.0,0.000309,32316.0
1,19001.0,4.0,0.000559,7152.0
2,21001.0,0.0,0.000000,19202.0
3,29001.0,10.0,0.000395,25343.0
4,40001.0,5.0,0.000225,22194.0
...,...,...,...,...
1614,48503.0,8.0,0.000444,18010.0
1615,2290.0,0.0,0.000000,5230.0
1616,8125.0,6.0,0.000599,10019.0
1617,48507.0,0.0,0.000000,11840.0


----

# Proportion of Race

In [18]:
all_county_race = pd.read_csv('race.csv')
all_county_race.head()

,proportionAIAN,proportionAsian,proportionBlack,proportionNHPI,proportionTwoOrMore,proportionWhite,parent.id
0,0.003376,0.004189,0.276610,0.000203,0.013910,0.701712,Abbeville_SouthCarolina_UnitedStates
1,0.003874,0.003134,0.181824,0.000305,0.015768,0.795094,Acadia_Louisiana_UnitedStates
2,0.006937,0.007980,0.285995,0.001596,0.017433,0.680059,Accomack_Virginia_UnitedStates
3,0.008042,0.027221,0.013533,0.002278,0.029117,0.919808,Ada_Idaho_UnitedStates
4,0.002545,0.005089,0.004948,0.000424,0.009613,0.977382,Adair_Iowa_UnitedStates


In [20]:
## Isolating rural counties
rural_race = all_county_race.loc[all_county_race['parent.id'].isin(list(rural['c3_data_id']))]
rural_race

,proportionAIAN,proportionAsian,proportionBlack,proportionNHPI,proportionTwoOrMore,proportionWhite,parent.id
2,0.006937,0.007980,0.285995,0.001596,0.017433,0.680059,Accomack_Virginia_UnitedStates
4,0.002545,0.005089,0.004948,0.000424,0.009613,0.977382,Adair_Iowa_UnitedStates
5,0.002813,0.003854,0.029533,0.000417,0.014063,0.949320,Adair_Kentucky_UnitedStates
6,0.003415,0.025279,0.031991,0.000667,0.019509,0.919140,Adair_Missouri_UnitedStates
7,0.460448,0.007964,0.006120,0.000720,0.102817,0.421931,Adair_Oklahoma_UnitedStates
...,...,...,...,...,...,...,...
3134,0.012565,0.007784,0.015679,0.000667,0.014789,0.948516,Young_Texas_UnitedStates
3136,0.699011,0.004665,0.009890,0.001306,0.056727,0.228401,Yukon-Koyukuk_Alaska_UnitedStates
3138,0.009048,0.005127,0.006032,0.002111,0.011159,0.966523,Yuma_Colorado_UnitedStates
3140,0.011279,0.002506,0.012616,0.001587,0.007018,0.964993,Zavala_Texas_UnitedStates


In [24]:
for i in range(1619):
    if rural['c3_data_id'][i] not in list(rural_race['parent.id']):
        print(rural['c3_data_id'][i])

O'Brien_Iowa_UnitedStates
St.Clair_Missouri_UnitedStates


---

# Social Distancing Grade (Outside Source: Unacast)

In [55]:
unacast_all = pd.read_csv('./rural_resurgence/unacast/unacast.csv')
## confirming only all rural counties present; previously sorted so just making sure
if unacast_all.countyFips.nunique() == rural.fips_code.nunique():
    print('Yay! All Rural counties!')
    
unacast_all

Yay! All Rural counties!


,FIPS,stateFips,stateName,stateCode,countyFips,countyName,date,totalGrade
0,1,1.0,Alabama,AL,1025.0,Clarke County,2020-11-08,D+
1,1,1.0,Alabama,AL,1025.0,Clarke County,2020-11-07,D
2,1,1.0,Alabama,AL,1025.0,Clarke County,2020-11-06,D
3,1,1.0,Alabama,AL,1025.0,Clarke County,2020-11-05,D
4,1,1.0,Alabama,AL,1025.0,Clarke County,2020-11-04,D
...,...,...,...,...,...,...,...,...
422390,56,56.0,Wyoming,WY,56043.0,Washakie County,2020-02-28,C
422391,56,56.0,Wyoming,WY,56043.0,Washakie County,2020-02-27,C
422392,56,56.0,Wyoming,WY,56043.0,Washakie County,2020-02-26,C
422393,56,56.0,Wyoming,WY,56043.0,Washakie County,2020-02-25,C


### Mapping totalGrade Letter Score to a Numeric Value

Originally, the scale from Unacast was from 1-5 with lower scores corresponding to a better social distance grade and so we mapped a similar scale to the totalGrade column for comparison and average grading. We changed the scale from A = 1, B = 2, C = 3, D = 4, F =5 to map it to a more consistent scale to accommodate partial scores (+ and -). Modeled after GPA scaling.

In [56]:
SDG_dict = {"F":5, "D-":4.7,"D": 4.3, "D+":4, "C-":3.7, "C": 3.3, "C+":3, "B-":2.7, "B": 2.3 , "B+":2, "A-":1.7, "A":1, "A+": 1}

In [57]:
unacast_all['grade_num'] = unacast_all['totalGrade'].map(SDG_dict)
unacast_all

,FIPS,stateFips,stateName,stateCode,countyFips,countyName,date,totalGrade,grade_num
0,1,1.0,Alabama,AL,1025.0,Clarke County,2020-11-08,D+,4.0
1,1,1.0,Alabama,AL,1025.0,Clarke County,2020-11-07,D,4.3
2,1,1.0,Alabama,AL,1025.0,Clarke County,2020-11-06,D,4.3
3,1,1.0,Alabama,AL,1025.0,Clarke County,2020-11-05,D,4.3
4,1,1.0,Alabama,AL,1025.0,Clarke County,2020-11-04,D,4.3
...,...,...,...,...,...,...,...,...,...
422390,56,56.0,Wyoming,WY,56043.0,Washakie County,2020-02-28,C,3.3
422391,56,56.0,Wyoming,WY,56043.0,Washakie County,2020-02-27,C,3.3
422392,56,56.0,Wyoming,WY,56043.0,Washakie County,2020-02-26,C,3.3
422393,56,56.0,Wyoming,WY,56043.0,Washakie County,2020-02-25,C,3.3


### Monthly Social Distance Grade for Each Rural County

In [334]:
import datetime

In [385]:
# date is formatted as string so reformatted to datetime 
unacast_all['month'] = pd.to_datetime(unacast_all['date'], format='%y/%m/%d').dt.month

# groupby countyFips and date
unacast_month = unacast_all.groupby([unacast_all["countyFips"], unacast_all['month']])['grade_num'].mean()
sdg_month = pd.DataFrame(unacast_month)
sdg_month.index = sdg_month.index.set_names(['countyFips', 'month'])
sdg_month

grade_num
countyFips month           
1005.0     2.0     4.750000
           3.0     4.441935
           4.0     4.040000
           5.0     4.748387
           6.0     4.743333
...                     ...
56045.0    7.0     3.269697
           8.0     3.261290
           9.0     3.100000
           10.0    3.022581
           11.0    3.175000

[16190 rows x 1 columns]

#### What about the Null Values?

Filling them with the monthly averages from all the counties

In [386]:
average_sdg_per_month = unacast_all.groupby(unacast_all['month'])['grade_num'].mean()
average_sdg_per_month

month
2.0     3.958491
3.0     3.623934
4.0     3.237983
5.0     3.709665
6.0     3.881027
7.0     3.903897
8.0     3.879737
9.0     3.787422
10.0    3.727534
11.0    3.622435
Name: grade_num, dtype: float64

In [387]:
month_avg = dict(average_sdg_per_month)

### Backfilling Holes

In [388]:
sdg_month.index

MultiIndex([( 1005.0,  2.0),
            ( 1005.0,  3.0),
            ( 1005.0,  4.0),
            ( 1005.0,  5.0),
            ( 1005.0,  6.0),
            ( 1005.0,  7.0),
            ( 1005.0,  8.0),
            ( 1005.0,  9.0),
            ( 1005.0, 10.0),
            ( 1005.0, 11.0),
            ...
            (56045.0,  2.0),
            (56045.0,  3.0),
            (56045.0,  4.0),
            (56045.0,  5.0),
            (56045.0,  6.0),
            (56045.0,  7.0),
            (56045.0,  8.0),
            (56045.0,  9.0),
            (56045.0, 10.0),
            (56045.0, 11.0)],
           names=['countyFips', 'month'], length=16190)

In [389]:
sdg_month.reset_index(inplace = True)
sdg_month

,countyFips,month,grade_num
0,1005.0,2.0,4.750000
1,1005.0,3.0,4.441935
2,1005.0,4.0,4.040000
3,1005.0,5.0,4.748387
4,1005.0,6.0,4.743333
...,...,...,...
16185,56045.0,7.0,3.269697
16186,56045.0,8.0,3.261290
16187,56045.0,9.0,3.100000
16188,56045.0,10.0,3.022581


In [390]:
sdg_month['month_avg'] = sdg_month.month.map(month_avg)

In [391]:
sdg_month

,countyFips,month,grade_num,month_avg
0,1005.0,2.0,4.750000,3.958491
1,1005.0,3.0,4.441935,3.623934
2,1005.0,4.0,4.040000,3.237983
3,1005.0,5.0,4.748387,3.709665
4,1005.0,6.0,4.743333,3.881027
...,...,...,...,...
16185,56045.0,7.0,3.269697,3.903897
16186,56045.0,8.0,3.261290,3.879737
16187,56045.0,9.0,3.100000,3.787422
16188,56045.0,10.0,3.022581,3.727534


In [392]:
sdg_month['grade_num'] = sdg_month['grade_num'].fillna(sdg_month['month_avg'])
sdg_month

,countyFips,month,grade_num,month_avg
0,1005.0,2.0,4.750000,3.958491
1,1005.0,3.0,4.441935,3.623934
2,1005.0,4.0,4.040000,3.237983
3,1005.0,5.0,4.748387,3.709665
4,1005.0,6.0,4.743333,3.881027
...,...,...,...,...
16185,56045.0,7.0,3.269697,3.903897
16186,56045.0,8.0,3.261290,3.879737
16187,56045.0,9.0,3.100000,3.787422
16188,56045.0,10.0,3.022581,3.727534


In [393]:
sdg_month.isna().sum()

countyFips    0
month         0
grade_num     0
month_avg     0
dtype: int64

In [394]:
sdg_month

,countyFips,month,grade_num,month_avg
0,1005.0,2.0,4.750000,3.958491
1,1005.0,3.0,4.441935,3.623934
2,1005.0,4.0,4.040000,3.237983
3,1005.0,5.0,4.748387,3.709665
4,1005.0,6.0,4.743333,3.881027
...,...,...,...,...
16185,56045.0,7.0,3.269697,3.903897
16186,56045.0,8.0,3.261290,3.879737
16187,56045.0,9.0,3.100000,3.787422
16188,56045.0,10.0,3.022581,3.727534


In [396]:
sdg_pivot = sdg_month.pivot(index = "countyFips", columns = "month", values = "grade_num")
sdg_pivot

month,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0
countyFips,,,,,,,,,,
1005.0,4.750000,4.441935,4.040000,4.748387,4.743333,4.745455,4.509677,4.430000,4.338710,4.2625
1011.0,3.366667,3.164516,2.960000,3.261290,3.313333,3.300000,3.280645,3.280000,3.274194,3.3000
1013.0,4.783333,4.261290,3.596667,4.370968,4.713333,4.748485,4.470968,4.386667,4.348387,4.3000
1019.0,5.000000,4.600000,4.166667,4.880645,5.000000,5.000000,5.000000,4.900000,4.806452,4.7250
1023.0,4.300000,4.074194,3.676667,3.980645,4.110000,4.133333,4.167742,4.120000,4.074194,4.0375
...,...,...,...,...,...,...,...,...,...,...
56037.0,4.000000,3.967742,3.936667,4.000000,4.000000,4.000000,4.000000,3.976667,3.977419,4.0000
56039.0,3.300000,2.703226,1.573333,2.529032,3.460000,3.700000,3.429032,3.300000,3.280645,2.9000
56041.0,4.000000,3.525806,2.973333,3.861290,4.000000,4.054545,4.029032,3.990000,3.990323,3.9625


In [405]:
month_dict={1:'sdg_January', 2:'sdg_February', 3:'sdg_March', 4:'sdg_April', 5:'sdg_May', 6:'sdg_June', 
           7:'sdg_July', 8:'sdg_August', 9:'sdg_September', 10:'sdg_October', 11:'sdg_November', 12:'sdg_December'}
sdg_month_final = sdg_pivot.rename(columns = month_dict)
sdg_month_final

month,sdg_February,sdg_March,sdg_April,sdg_May,sdg_June,sdg_July,sdg_August,sdg_September,sdg_October,sdg_November
countyFips,,,,,,,,,,
1005.0,4.750000,4.441935,4.040000,4.748387,4.743333,4.745455,4.509677,4.430000,4.338710,4.2625
1011.0,3.366667,3.164516,2.960000,3.261290,3.313333,3.300000,3.280645,3.280000,3.274194,3.3000
1013.0,4.783333,4.261290,3.596667,4.370968,4.713333,4.748485,4.470968,4.386667,4.348387,4.3000
1019.0,5.000000,4.600000,4.166667,4.880645,5.000000,5.000000,5.000000,4.900000,4.806452,4.7250
1023.0,4.300000,4.074194,3.676667,3.980645,4.110000,4.133333,4.167742,4.120000,4.074194,4.0375
...,...,...,...,...,...,...,...,...,...,...
56037.0,4.000000,3.967742,3.936667,4.000000,4.000000,4.000000,4.000000,3.976667,3.977419,4.0000
56039.0,3.300000,2.703226,1.573333,2.529032,3.460000,3.700000,3.429032,3.300000,3.280645,2.9000
56041.0,4.000000,3.525806,2.973333,3.861290,4.000000,4.054545,4.029032,3.990000,3.990323,3.9625


# Final CSV of Data

Have: 
* rural counties key (rural df)
* Proportion of High Risk (rural_conditions_proportion df)
* SDG (sdg_month df)

In [406]:
rural_cond = rural.merge(rural_conditions_proportion, left_on='fips_code', right_on = "FIPS")
rural_cond

,fips_code,county,state,c3_data_id,COUNTY_NAME,STATE_NAME,FIPS,proportionAnyCondition
0,51001.0,Accomack,Virginia,Accomack_Virginia_UnitedStates,Accomack,Virginia,51001,0.499961
1,19001.0,Adair,Iowa,Adair_Iowa_UnitedStates,Adair,Iowa,19001,0.436856
2,21001.0,Adair,Kentucky,Adair_Kentucky_UnitedStates,Adair,Kentucky,21001,0.499837
3,29001.0,Adair,Missouri,Adair_Missouri_UnitedStates,Adair,Missouri,29001,0.413798
4,40001.0,Adair,Oklahoma,Adair_Oklahoma_UnitedStates,Adair,Oklahoma,40001,0.596188
...,...,...,...,...,...,...,...,...
1614,48503.0,Young,Texas,Young_Texas_UnitedStates,Young,Texas,48503,0.507146
1615,2290.0,Yukon-Koyukuk,Alaska,Yukon-Koyukuk_Alaska_UnitedStates,Yukon-Koyukuk,Alaska,2290,0.473793
1616,8125.0,Yuma,Colorado,Yuma_Colorado_UnitedStates,Yuma,Colorado,8125,0.358638
1617,48507.0,Zavala,Texas,Zavala_Texas_UnitedStates,Zavala,Texas,48507,0.567064


In [407]:
w_sdg = rural_cond.merge(sdg_month_final, left_on='fips_code', right_on = "countyFips")
final_sdg = w_sdg.iloc[:, [0,1,2,3,7,8,9,10,11,12,13,14,15,16,17]]
final_sdg

,fips_code,county,state,c3_data_id,proportionAnyCondition,sdg_February,sdg_March,sdg_April,sdg_May,sdg_June,sdg_July,sdg_August,sdg_September,sdg_October,sdg_November
0,51001.0,Accomack,Virginia,Accomack_Virginia_UnitedStates,0.499961,4.300000,4.154839,3.483333,4.167742,4.693333,4.972727,4.635484,4.320000,4.106452,4.000000
1,19001.0,Adair,Iowa,Adair_Iowa_UnitedStates,0.436856,3.700000,3.429032,3.240000,3.403226,3.700000,3.718182,3.693548,3.600000,3.403226,3.300000
2,21001.0,Adair,Kentucky,Adair_Kentucky_UnitedStates,0.499837,4.900000,4.332258,3.663333,4.203226,4.433333,4.430303,4.551613,4.326667,4.270968,4.112500
3,29001.0,Adair,Missouri,Adair_Missouri_UnitedStates,0.413798,5.000000,4.367742,3.773333,4.622581,4.610000,4.609091,4.841935,4.630000,4.322581,4.150000
4,40001.0,Adair,Oklahoma,Adair_Oklahoma_UnitedStates,0.596188,3.633333,3.235484,2.826667,3.312903,3.673333,3.693939,3.564516,3.353333,3.241935,3.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1614,48503.0,Young,Texas,Young_Texas_UnitedStates,0.507146,4.633333,4.322581,3.990000,4.383871,4.613333,4.530303,4.609677,4.383333,4.312903,4.262500
1615,2290.0,Yukon-Koyukuk,Alaska,Yukon-Koyukuk_Alaska_UnitedStates,0.473793,3.958491,3.623934,3.237983,3.709665,3.881027,3.903897,3.879737,3.787422,3.727534,3.622435
1616,8125.0,Yuma,Colorado,Yuma_Colorado_UnitedStates,0.358638,3.300000,3.151613,3.100000,3.267742,3.300000,3.300000,3.300000,3.300000,3.280645,3.300000
1617,48507.0,Zavala,Texas,Zavala_Texas_UnitedStates,0.567064,3.300000,3.177419,2.786667,3.164516,3.220000,2.990909,3.016129,3.166667,3.300000,3.225000


----

# Monthly Cases - Cumulative and New Cases

In [408]:
monthly_cumulative = pd.read_csv('monthly_rural_cumulative_cases.csv')
monthly_new = pd.read_csv('monthly_rural_new_cases.csv')

In [409]:
monthly_cumulative

,fips_code,county,state,c3_data_id,feb_cases,mar_cases,apr_cases,may_cases,june_cases,july_cases,aug_cases,sept_cases,oct_cases,nov_cases
0,51001.0,Accomack,Virginia,Accomack_Virginia_UnitedStates,0.0,7.0,264.0,909.0,1042.0,1074.0,1145.0,1182.0,1223.0,1248.0
1,19001.0,Adair,Iowa,Adair_Iowa_UnitedStates,0.0,1.0,1.0,9.0,15.0,20.0,49.0,76.0,220.0,329.0
2,21001.0,Adair,Kentucky,Adair_Kentucky_UnitedStates,0.0,0.0,81.0,97.0,117.0,201.0,260.0,316.0,455.0,570.0
3,29001.0,Adair,Missouri,Adair_Missouri_UnitedStates,0.0,2.0,12.0,55.0,90.0,132.0,229.0,350.0,519.0,817.0
4,40001.0,Adair,Oklahoma,Adair_Oklahoma_UnitedStates,0.0,4.0,64.0,87.0,115.0,283.0,447.0,639.0,850.0,971.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1614,48503.0,Young,Texas,Young_Texas_UnitedStates,0.0,1.0,4.0,4.0,24.0,133.0,244.0,332.0,497.0,704.0
1615,2290.0,Yukon-Koyukuk,Alaska,Yukon-Koyukuk_Alaska_UnitedStates,0.0,0.0,1.0,1.0,3.0,45.0,71.0,81.0,120.0,132.0
1616,8125.0,Yuma,Colorado,Yuma_Colorado_UnitedStates,0.0,2.0,8.0,35.0,52.0,62.0,63.0,125.0,227.0,262.0
1617,48507.0,Zavala,Texas,Zavala_Texas_UnitedStates,0.0,0.0,1.0,11.0,21.0,170.0,261.0,369.0,463.0,498.0


In [410]:
monthly_new

,fips_code,county,state,c3_data_id,feb_new,mar_new,apr_new,may_new,june_new,july_new,aug_new,sept_new,oct_new,nov_new
0,51001.0,Accomack,Virginia,Accomack_Virginia_UnitedStates,0.0,7.0,257.0,645.0,133.0,32.0,71.0,37.0,41.0,25.0
1,19001.0,Adair,Iowa,Adair_Iowa_UnitedStates,0.0,1.0,0.0,8.0,6.0,5.0,29.0,27.0,144.0,109.0
2,21001.0,Adair,Kentucky,Adair_Kentucky_UnitedStates,0.0,0.0,81.0,16.0,20.0,84.0,59.0,56.0,139.0,115.0
3,29001.0,Adair,Missouri,Adair_Missouri_UnitedStates,0.0,2.0,10.0,43.0,35.0,42.0,97.0,121.0,169.0,298.0
4,40001.0,Adair,Oklahoma,Adair_Oklahoma_UnitedStates,0.0,4.0,60.0,23.0,28.0,168.0,164.0,192.0,211.0,121.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1614,48503.0,Young,Texas,Young_Texas_UnitedStates,0.0,1.0,3.0,0.0,20.0,109.0,111.0,88.0,165.0,207.0
1615,2290.0,Yukon-Koyukuk,Alaska,Yukon-Koyukuk_Alaska_UnitedStates,0.0,0.0,1.0,0.0,2.0,42.0,26.0,10.0,39.0,12.0
1616,8125.0,Yuma,Colorado,Yuma_Colorado_UnitedStates,0.0,2.0,6.0,27.0,17.0,10.0,1.0,62.0,102.0,35.0
1617,48507.0,Zavala,Texas,Zavala_Texas_UnitedStates,0.0,0.0,1.0,10.0,10.0,149.0,91.0,108.0,94.0,35.0


In [411]:
cumulative_cases = final_sdg.merge(monthly_cumulative, on='fips_code')
cumulative_cases

,fips_code,county_x,state_x,c3_data_id_x,proportionAnyCondition,sdg_February,sdg_March,sdg_April,sdg_May,sdg_June,...,feb_cases,mar_cases,apr_cases,may_cases,june_cases,july_cases,aug_cases,sept_cases,oct_cases,nov_cases
0,51001.0,Accomack,Virginia,Accomack_Virginia_UnitedStates,0.499961,4.300000,4.154839,3.483333,4.167742,4.693333,...,0.0,7.0,264.0,909.0,1042.0,1074.0,1145.0,1182.0,1223.0,1248.0
1,19001.0,Adair,Iowa,Adair_Iowa_UnitedStates,0.436856,3.700000,3.429032,3.240000,3.403226,3.700000,...,0.0,1.0,1.0,9.0,15.0,20.0,49.0,76.0,220.0,329.0
2,21001.0,Adair,Kentucky,Adair_Kentucky_UnitedStates,0.499837,4.900000,4.332258,3.663333,4.203226,4.433333,...,0.0,0.0,81.0,97.0,117.0,201.0,260.0,316.0,455.0,570.0
3,29001.0,Adair,Missouri,Adair_Missouri_UnitedStates,0.413798,5.000000,4.367742,3.773333,4.622581,4.610000,...,0.0,2.0,12.0,55.0,90.0,132.0,229.0,350.0,519.0,817.0
4,40001.0,Adair,Oklahoma,Adair_Oklahoma_UnitedStates,0.596188,3.633333,3.235484,2.826667,3.312903,3.673333,...,0.0,4.0,64.0,87.0,115.0,283.0,447.0,639.0,850.0,971.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1614,48503.0,Young,Texas,Young_Texas_UnitedStates,0.507146,4.633333,4.322581,3.990000,4.383871,4.613333,...,0.0,1.0,4.0,4.0,24.0,133.0,244.0,332.0,497.0,704.0
1615,2290.0,Yukon-Koyukuk,Alaska,Yukon-Koyukuk_Alaska_UnitedStates,0.473793,3.958491,3.623934,3.237983,3.709665,3.881027,...,0.0,0.0,1.0,1.0,3.0,45.0,71.0,81.0,120.0,132.0
1616,8125.0,Yuma,Colorado,Yuma_Colorado_UnitedStates,0.358638,3.300000,3.151613,3.100000,3.267742,3.300000,...,0.0,2.0,8.0,35.0,52.0,62.0,63.0,125.0,227.0,262.0
1617,48507.0,Zavala,Texas,Zavala_Texas_UnitedStates,0.567064,3.300000,3.177419,2.786667,3.164516,3.220000,...,0.0,0.0,1.0,11.0,21.0,170.0,261.0,369.0,463.0,498.0


In [412]:
new_cases = cumulative_cases.merge(monthly_new, on='fips_code')
new_cases

,fips_code,county_x,state_x,c3_data_id_x,proportionAnyCondition,sdg_February,sdg_March,sdg_April,sdg_May,sdg_June,...,feb_new,mar_new,apr_new,may_new,june_new,july_new,aug_new,sept_new,oct_new,nov_new
0,51001.0,Accomack,Virginia,Accomack_Virginia_UnitedStates,0.499961,4.300000,4.154839,3.483333,4.167742,4.693333,...,0.0,7.0,257.0,645.0,133.0,32.0,71.0,37.0,41.0,25.0
1,19001.0,Adair,Iowa,Adair_Iowa_UnitedStates,0.436856,3.700000,3.429032,3.240000,3.403226,3.700000,...,0.0,1.0,0.0,8.0,6.0,5.0,29.0,27.0,144.0,109.0
2,21001.0,Adair,Kentucky,Adair_Kentucky_UnitedStates,0.499837,4.900000,4.332258,3.663333,4.203226,4.433333,...,0.0,0.0,81.0,16.0,20.0,84.0,59.0,56.0,139.0,115.0
3,29001.0,Adair,Missouri,Adair_Missouri_UnitedStates,0.413798,5.000000,4.367742,3.773333,4.622581,4.610000,...,0.0,2.0,10.0,43.0,35.0,42.0,97.0,121.0,169.0,298.0
4,40001.0,Adair,Oklahoma,Adair_Oklahoma_UnitedStates,0.596188,3.633333,3.235484,2.826667,3.312903,3.673333,...,0.0,4.0,60.0,23.0,28.0,168.0,164.0,192.0,211.0,121.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1614,48503.0,Young,Texas,Young_Texas_UnitedStates,0.507146,4.633333,4.322581,3.990000,4.383871,4.613333,...,0.0,1.0,3.0,0.0,20.0,109.0,111.0,88.0,165.0,207.0
1615,2290.0,Yukon-Koyukuk,Alaska,Yukon-Koyukuk_Alaska_UnitedStates,0.473793,3.958491,3.623934,3.237983,3.709665,3.881027,...,0.0,0.0,1.0,0.0,2.0,42.0,26.0,10.0,39.0,12.0
1616,8125.0,Yuma,Colorado,Yuma_Colorado_UnitedStates,0.358638,3.300000,3.151613,3.100000,3.267742,3.300000,...,0.0,2.0,6.0,27.0,17.0,10.0,1.0,62.0,102.0,35.0
1617,48507.0,Zavala,Texas,Zavala_Texas_UnitedStates,0.567064,3.300000,3.177419,2.786667,3.164516,3.220000,...,0.0,0.0,1.0,10.0,10.0,149.0,91.0,108.0,94.0,35.0


### Adding In Unemployment and Medical Preparedness Metrics

In [473]:
med_prep = new_cases.merge(rural_icus, on='fips_code')
full_data = med_prep.merge(rural_unemp_df, on = 'fips_code')
full_data

,fips_code,county_x,state_x,c3_data_id_x,proportionAnyCondition,sdg_February,sdg_March,sdg_April,sdg_May,sdg_June,...,june_new,july_new,aug_new,sept_new,oct_new,nov_new,hospitalIcuBeds,icu_beds_per_capita,populationCDS,latestUnemploymentRate
0,51001.0,Accomack,Virginia,Accomack_Virginia_UnitedStates,0.499961,4.300000,4.154839,3.483333,4.167742,4.693333,...,133.0,32.0,71.0,37.0,41.0,25.0,10.0,0.000309,32316.0,5.655897
1,19001.0,Adair,Iowa,Adair_Iowa_UnitedStates,0.436856,3.700000,3.429032,3.240000,3.403226,3.700000,...,6.0,5.0,29.0,27.0,144.0,109.0,4.0,0.000559,7152.0,2.977732
2,21001.0,Adair,Kentucky,Adair_Kentucky_UnitedStates,0.499837,4.900000,4.332258,3.663333,4.203226,4.433333,...,20.0,84.0,59.0,56.0,139.0,115.0,0.0,0.000000,19202.0,5.503603
3,29001.0,Adair,Missouri,Adair_Missouri_UnitedStates,0.413798,5.000000,4.367742,3.773333,4.622581,4.610000,...,35.0,42.0,97.0,121.0,169.0,298.0,10.0,0.000395,25343.0,2.987000
4,40001.0,Adair,Oklahoma,Adair_Oklahoma_UnitedStates,0.596188,3.633333,3.235484,2.826667,3.312903,3.673333,...,28.0,168.0,164.0,192.0,211.0,121.0,5.0,0.000225,22194.0,4.933655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1614,48503.0,Young,Texas,Young_Texas_UnitedStates,0.507146,4.633333,4.322581,3.990000,4.383871,4.613333,...,20.0,109.0,111.0,88.0,165.0,207.0,8.0,0.000444,18010.0,5.745519
1615,2290.0,Yukon-Koyukuk,Alaska,Yukon-Koyukuk_Alaska_UnitedStates,0.473793,3.958491,3.623934,3.237983,3.709665,3.881027,...,2.0,42.0,26.0,10.0,39.0,12.0,0.0,0.000000,5230.0,8.801415
1616,8125.0,Yuma,Colorado,Yuma_Colorado_UnitedStates,0.358638,3.300000,3.151613,3.100000,3.267742,3.300000,...,17.0,10.0,1.0,62.0,102.0,35.0,6.0,0.000599,10019.0,2.834331
1617,48507.0,Zavala,Texas,Zavala_Texas_UnitedStates,0.567064,3.300000,3.177419,2.786667,3.164516,3.220000,...,10.0,149.0,91.0,108.0,94.0,35.0,0.0,0.000000,11840.0,16.953762


In [474]:
key = list(full_data.columns)
value = [i for i in range(len(key))]
col_dict = {k:v for (k,v) in zip(key,value)}

In [475]:
final_columns = ['fips_code', 'county', 'state', 'c3_data_id', 'populationCDS',
       'latestUnemploymentRate','hospitalIcuBeds', 'icu_beds_per_capita',
       'proportionAnyCondition', 'sdg_February', 'sdg_March', 'sdg_April',
       'sdg_May', 'sdg_June', 'sdg_July', 'sdg_August', 'sdg_September','sdg_October', 'sdg_November', 
       'feb_cases', 'mar_cases', 'apr_cases', 'may_cases', 'june_cases',
       'july_cases', 'aug_cases', 'sept_cases', 'oct_cases', 'nov_cases','feb_new', 'mar_new', 'apr_new',
       'may_new', 'june_new', 'july_new', 'aug_new', 'sept_new', 'oct_new',
       'nov_new']

In [476]:
final_data_monthly = full_data.iloc[:, [col_dict[i] for i in final_columns]]
final_data_monthly

,fips_code,county,state,c3_data_id,populationCDS,latestUnemploymentRate,hospitalIcuBeds,icu_beds_per_capita,proportionAnyCondition,sdg_February,...,feb_new,mar_new,apr_new,may_new,june_new,july_new,aug_new,sept_new,oct_new,nov_new
0,51001.0,Accomack,Virginia,Accomack_Virginia_UnitedStates,32316.0,5.655897,10.0,0.000309,0.499961,4.300000,...,0.0,7.0,257.0,645.0,133.0,32.0,71.0,37.0,41.0,25.0
1,19001.0,Adair,Iowa,Adair_Iowa_UnitedStates,7152.0,2.977732,4.0,0.000559,0.436856,3.700000,...,0.0,1.0,0.0,8.0,6.0,5.0,29.0,27.0,144.0,109.0
2,21001.0,Adair,Kentucky,Adair_Kentucky_UnitedStates,19202.0,5.503603,0.0,0.000000,0.499837,4.900000,...,0.0,0.0,81.0,16.0,20.0,84.0,59.0,56.0,139.0,115.0
3,29001.0,Adair,Missouri,Adair_Missouri_UnitedStates,25343.0,2.987000,10.0,0.000395,0.413798,5.000000,...,0.0,2.0,10.0,43.0,35.0,42.0,97.0,121.0,169.0,298.0
4,40001.0,Adair,Oklahoma,Adair_Oklahoma_UnitedStates,22194.0,4.933655,5.0,0.000225,0.596188,3.633333,...,0.0,4.0,60.0,23.0,28.0,168.0,164.0,192.0,211.0,121.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1614,48503.0,Young,Texas,Young_Texas_UnitedStates,18010.0,5.745519,8.0,0.000444,0.507146,4.633333,...,0.0,1.0,3.0,0.0,20.0,109.0,111.0,88.0,165.0,207.0
1615,2290.0,Yukon-Koyukuk,Alaska,Yukon-Koyukuk_Alaska_UnitedStates,5230.0,8.801415,0.0,0.000000,0.473793,3.958491,...,0.0,0.0,1.0,0.0,2.0,42.0,26.0,10.0,39.0,12.0
1616,8125.0,Yuma,Colorado,Yuma_Colorado_UnitedStates,10019.0,2.834331,6.0,0.000599,0.358638,3.300000,...,0.0,2.0,6.0,27.0,17.0,10.0,1.0,62.0,102.0,35.0
1617,48507.0,Zavala,Texas,Zavala_Texas_UnitedStates,11840.0,16.953762,0.0,0.000000,0.567064,3.300000,...,0.0,0.0,1.0,10.0,10.0,149.0,91.0,108.0,94.0,35.0


In [477]:
final_data_monthly.to_csv('full_rural_monthly_data.csv', index = False)